In [1]:
from glob import glob
from pathlib import Path
from face_alignment import align
from inference import load_pretrained_model, to_input

architecture = 'ir_50'
pretrained_path = '/home/s3-subtitle-download/pretrained_models/AdaFace/adaface_ir50_ms1mv2.ckpt'
model = load_pretrained_model(architecture, pretrained_path)

### ONNX batch

In [ ]:
import torch
import onnxruntime as ort

ONNX_PATH = 'adaface.onnx'

paths = glob('_test/*.jpg')

faces = []
for path in paths:
    aligned_rgb_img = align.get_aligned_face(path)
    bgr_input = to_input(aligned_rgb_img)
    faces.append(bgr_input)

faces = torch.cat(faces, dim=0)

ort_session = ort.InferenceSession(ONNX_PATH)
input_name = ort_session.get_inputs()[0].name
input_data = faces.numpy()
features = ort_session.run(None, {input_name: input_data})
features = features[0]

names = [Path(path).stem for path in paths]
features = {name: torch.tensor(feature).unsqueeze(0) for name, feature in zip(names, features)}

### single image

In [3]:
# features = {}
# for path in glob('_test/*.jpg'):
#     aligned_rgb_img = align.get_aligned_face(path)
#     bgr_input = to_input(aligned_rgb_img)
#     feature, _ = model(bgr_input)
#     features[Path(path).stem] = feature

### multiple images, bath processing

In [4]:
# # multiple images, bath processing
# import torch
# paths = glob('_test/*.jpg')

# faces = []
# for path in paths:
#     aligned_rgb_img = align.get_aligned_face(path)
#     bgr_input = to_input(aligned_rgb_img)
#     faces.append(bgr_input)

# faces = torch.cat(faces, dim=0)
# features = model(faces)[0]

# names = [Path(path).stem for path in paths]
# features = {name: feature.unsqueeze(0) for name, feature in zip(names, features)}

In [5]:
# calculate similarity
import torch.nn.functional as F
for name1, feature1 in features.items():
    for name2, feature2 in features.items():
        print(f'{name1:8}, {name2:8}\t{F.cosine_similarity(feature1, feature2).item():.4f}')

hasegawa, hasegawa	1.0000
hasegawa, taka    	-0.0315
hasegawa, watanabe	0.0633
hasegawa, toshi   	-0.0307
taka    , hasegawa	-0.0315
taka    , taka    	1.0000
taka    , watanabe	-0.0136
taka    , toshi   	0.1365
watanabe, hasegawa	0.0633
watanabe, taka    	-0.0136
watanabe, watanabe	1.0000
watanabe, toshi   	-0.0458
toshi   , hasegawa	-0.0307
toshi   , taka    	0.1365
toshi   , watanabe	-0.0458
toshi   , toshi   	1.0000


# Comparison to FaceRecognition

In [6]:
import torch
import face_recognition
fr_features = {}
for path in glob('_test/*.jpg'):
    img = face_recognition.load_image_file(path)
    ft = face_recognition.face_encodings(img)
    fr_features[Path(path).stem] = torch.tensor(ft)

for name1, feature1 in fr_features.items():
    for name2, feature2 in fr_features.items():
        print(f'{name1:8}, {name2:8}\t{F.cosine_similarity(feature1, feature2).item():.4f}')

hasegawa, hasegawa	1.0000
hasegawa, taka    	0.8935
hasegawa, watanabe	0.8884
hasegawa, toshi   	0.8968
taka    , hasegawa	0.8935
taka    , taka    	1.0000
taka    , watanabe	0.9072
taka    , toshi   	0.9405
watanabe, hasegawa	0.8884
watanabe, taka    	0.9072
watanabe, watanabe	1.0000
watanabe, toshi   	0.9070
toshi   , hasegawa	0.8968
toshi   , taka    	0.9405
toshi   , watanabe	0.9070
toshi   , toshi   	1.0000
